In [0]:


ALTER table preprossesed_mines_data add columns (shift_begin_ts timestamp, accident_ts timestamp ,shift_type string);

-- Update the new columns with the appropriate values
UPDATE preprossesed_mines_data
SET 
    shift_begin_ts = try_to_timestamp(
      concat(
        '1970-01-01 ',
        substr(lpad(shift_begin_time, 4, '0'), 1, 2), ':', substr(lpad(shift_begin_time, 4, '0'), 3, 2)
      ),
      'yyyy-MM-dd HH:mm'
    ),
    accident_ts = try_to_timestamp(
      concat(
        '1970-01-01 ',
        substr(lpad(accident_time, 4, '0'), 1, 2), ':', substr(lpad(accident_time, 4, '0'), 3, 2)
      ),
      'yyyy-MM-dd HH:mm'
    ),
    shift_type = CASE
      WHEN hour(try_to_timestamp(
        concat(
          '1970-01-01 ',
          substr(lpad(shift_begin_time, 4, '0'), 1, 2), ':', substr(lpad(shift_begin_time, 4, '0'), 3, 2)
        ),
        'yyyy-MM-dd HH:mm'
      )) >= 18 -- 6 PM
      OR hour(try_to_timestamp(
        concat(
          '1970-01-01 ',
          substr(lpad(shift_begin_time, 4, '0'), 1, 2), ':', substr(lpad(shift_begin_time, 4, '0'), 3, 2)
        ),
        'yyyy-MM-dd HH:mm'
      )) < 6 -- before 6 AM
      THEN 'Night Shift'
      ELSE 'Day Shift'
    END;

ALTER table combined_mine_data add columns (staton_id String, station_name String,distance_km double, tavg double,tmin double,tmax double,
prcp double,snow double,wdir double,wspd double,pres double,tsun double,weather_summary String)

-- Update combined_mine_data with data from mine_accident_weather
MERGE INTO combined_mine_data AS target
USING mine_accident_weather AS source
ON target.current_mine_name = source.current_mine_name and DATE(target.accident_dt) = Date(source.accident_dt) 
and target.latitude = source.latitude and target.longitude = source.longitude  

WHEN MATCHED THEN
  UPDATE SET
    target.staton_id = source.station_id,
    target.station_name = source.station_name,
    target.distance_km = source.distance_km,
    target.tavg = source.tavg,
    target.tmin = source.tmin,
    target.tmax = source.tmax,
    target.prcp = source.prcp,
    target.snow = source.snow,
    target.wdir = source.wdir,
    target.wspd = source.wspd,
    target.pres = source.pres,
    target.tsun = source.tsun,
    target.weather_summary = source.weather_summary;



  select mine_id,current_mine_type,shift_type,accident_dt, ROUND(
        (unix_timestamp(accident_ts) - unix_timestamp(shift_begin_ts)) / 3600.0, 2) as time_diff, 
        activity, weather_summary , accident_type
  from preprossesed_mines_data where mine_id = 1302256
  order by accident_dt



  select a.mine_id,b.* from combined_mine_data a  
  inner join mine_accident_weather b on a.current_mine_name = b.current_mine_name 
  and a.accident_dt = b.accident_dt and a.latitude = b.latitude and a.longitude = b.longitude
  where a.mine_id = 1302256
  limit 100

select * from combined_mine_data where mine_id = 1302256

select * from preprossesed_mines_data where mine_id = 1302256

drop table if exists preprossesed_mines_data
CREATE OR REPLACE TABLE preprossesed_mines_data AS
SELECT
c.*,
    w.station_id,
    w.station_name,
    w.distance_km,
    w.tavg,
    w.tmin,
    w.tmax,
    w.prcp,
    w.snow,
    w.wdir,
    w.wspd,
    w.pres,
    w.tsun,
    w.weather_summary
FROM combined_mine_data c
LEFT JOIN mine_accident_weather w
ON c.mine_id = w.mine_id
AND c.latitude = w.latitude
AND c.longitude = w.longitude
AND DATE(c.accident_dt) = Date(w.accident_dt);

select distinct hours_per_shift from mine_locations
where mine_id = 1302256

select * from mine_accidents where mine_id = 1302256

alter table preprossesed_mines_data add column (time_diff double)
update preprossesed_mines_data
set time_diff = round((unix_timestamp(accident_ts) - unix_timestamp(shift_begin_ts)) / 3600.0, 2)


-- checking unity catalog enablement
SELECT CURRENT_METASTORE();

select * from mine_locations where mine_id = 1302256
---94.453333	41.305

select * from mine_accident_weather 
where longitude = -94.453333 and latitude = 41.305 and mine_id = 1302256

select * from preprossesed_mines_data where mine_id = 1302256

select * from combined_mine_data where mine_id = 1302256


